<a href="https://colab.research.google.com/github/JaswanthMannem/CREWAI/blob/main/crewai_6_job_application_crew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [ ]:
!pip install langchain-openai

In [3]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [4]:
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI

In [5]:
from google.colab import userdata
openai_key=userdata.get('OPENAI_API_KEY')
serp_key=userdata.get('SERP_API_KEY')

In [6]:
import os
os.environ["OPENAI_API_KEY"] = openai_key
openai_api_key = openai_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = serp_key

In [7]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./fake_resume.md')
semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md')

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  3.74it/s]


In [ ]:
from IPython.display import Markdown, display
display(Markdown("./fake_resume.md"))

In [10]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    ),
    llm=ChatOpenAI(openai_api_key=openai_api_key)
)

In [11]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    ),
    llm=ChatOpenAI(openai_api_key=openai_api_key)
)

In [12]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    ),
    llm=ChatOpenAI(openai_api_key=openai_api_key)
)

In [13]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    ),
    llm=ChatOpenAI(openai_api_key=openai_api_key)
)

In [14]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [15]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

In [17]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],  #this task will start after completing of tasks in context
    agent=resume_strategist
)

In [18]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


In [19]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

In [20]:
job_application_inputs = {
    'job_posting_url': '',
    'github_url': 'https://github.com/JaswanthMannem',
    'personal_writeup': """Jaswanth is an accomplished Software
    Engineering Leader with 3 months of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an B.Tech degree and a strong
    background in AI and data science. Jaswanth has successfully worked in
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

In [ ]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

In [ ]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

In [23]:
display(Markdown("./interview_materials.md"))

Based on Jaswanth Mannem's resume, here are some potential interview questions and talking points:

1. Can you walk us through your experience as a Speech and AI Intern at Ozonetel Communications? What were your main responsibilities and contributions to the team?
2. Tell us about a project you worked on that involved developing and optimizing speech recognition and NLP models using TensorFlow and Hugging Face. What challenges did you face and how did you overcome them?
3. How did you integrate open-source models with chatbots and RAG applications using Streamlit? Can you provide an example of a successful integration you worked on?
4. Discuss your experience with developing a medical chatbot using Python, Flask, LLaMA-2, Pinecone, and Lang-Chain. What was the goal of the project and how did you achieve it?
5. Explain the process of creating a MCQ Generator using openai and langchain. How did you ensure the accuracy and relevance of the generated questions?
6. Share details about your project on optimizing traffic sign detection and recognition using Python, TensorFlow, CNN, RNN, and LSTM. How did this project contribute to road safety and driver awareness?
7. Describe your work on the Food Chat-Bot with DialogFlow. How did you train the chatbot intents and connect it to the backend for handling online orders?
8. Can you elaborate on your technical skills in programming languages, toolkits, web frameworks, and platforms listed on your resume? Which tools are you most comfortable working with and why?
9. How have your certifications in Data Structures and Algorithms, Programming in Java, Neural Networks, and Deep Learning influenced your approach to problem-solving and project development?
10. Discuss your participation in workshops on Google Android Development with Kotlin and Blockchain Technology. How have these experiences broadened your skill set and knowledge in the tech industry?

These questions and talking points will help Jaswanth highlight his skills, experiences, and qualifications that align with the job requirements during the interview.